# Causal Graph Visualization

- Bokeh
- pivis

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import networkx as nx

from bokeh.io import output_notebook
from bokeh.plotting import figure, show, from_networkx

G = nx.karate_club_graph()
output_notebook()

Loading BokehJS ...

In [3]:
p = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools="", toolbar_location=None)

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
p.renderers.append(graph)
show(p)

In [4]:
from multiprocessing import cpu_count
import sys

sys.path.append('../')
from tsdr import tsdr
from diagnoser import diag
from meltria import loader
import diagnoser.metric_node as mn

filepath = '/datasets/argowf-chaos-hg68n/2021-12-09-argowf-chaos-hg68n-user_pod-memory-hog_0.json'
data_df, mappings = loader.read_metrics_file(filepath)

reducer = tsdr.Tsdr(tsdr.ar_based_ad_model, **{
    'tsifter_step1_ar_regression': 'n',
    'tsifter_step1_ar_anomaly_score_threshold': 0.01,
    'tsifter_step1_cv_threshold': 0.05,
    'tsifter_step1_ar_dynamic_prediction': False,
    'tsifter_step2_clustering_threshold': 0.01,
    'tsifter_step2_clustered_series_type': 'raw',
    'tsifter_step2_clustering_dist_type': 'sbd',
    'tsifter_step2_clustering_choice_method': 'medoid',
    'tsifter_step2_clustering_linkage_method': 'single',
})
_, reduced_df_by_step, metrics_dimension, _ = reducer.run(
    series=data_df,
    max_workers=cpu_count(),
)
reduced_df = reduced_df_by_step['step2']
causal_graph, stats = diag.run(
    reduced_df, mappings, **{
        'pc_library': 'pcalg',
        'pc_citest': 'fisher-z',
        'pc_citest_alpha': 0.01,
        'pc_variant': None,
    }
)
print(len(causal_graph.nodes))
relabeled_mapping = mn.MetricNodes.from_list_of_metric_node(list(causal_graph.nodes)).node_to_label()
print(relabeled_mapping)
relabeled_graph = nx.relabel_nodes(causal_graph, relabeled_mapping, copy=True)

16
{<diagnoser.metric_node.MetricNode object at 0x7f0080eb47c0>: 'c-carts_cpu_cfs_throttled_seconds_total', <diagnoser.metric_node.MetricNode object at 0x7f0080eb49a0>: 'c-front-end_memory_usage_bytes', <diagnoser.metric_node.MetricNode object at 0x7f0080eb4e80>: 'c-front-end_sockets', <diagnoser.metric_node.MetricNode object at 0x7f0080eb40d0>: 'c-front-end_network_receive_bytes_total', <diagnoser.metric_node.MetricNode object at 0x7f0080eaad00>: 'c-orders_cpu_cfs_throttled_seconds_total', <diagnoser.metric_node.MetricNode object at 0x7f0080eaa250>: 'c-orders_cpu_cfs_throttled_periods_total', <diagnoser.metric_node.MetricNode object at 0x7f0080eaa9d0>: 'c-orders_cpu_cfs_periods_total', <diagnoser.metric_node.MetricNode object at 0x7f0080eb2580>: 'c-user_sockets', <diagnoser.metric_node.MetricNode object at 0x7f0080eb2730>: 'c-user_threads', <diagnoser.metric_node.MetricNode object at 0x7f0080eb2be0>: 'c-user_last_seen', <diagnoser.metric_node.MetricNode object at 0x7f0080eb2880>: 's-c

In [5]:
from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool, NodesAndLinkedEdges)
from bokeh.palettes import Spectral4


HOVER_TOOLTIPS = [("Metric", "@index")]
p = figure(title="caulsal graph visualization", x_range=(-1.1,1.1), y_range=(-1.1,1.1),
            tooltips = HOVER_TOOLTIPS, tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom')
gr = from_networkx(relabeled_graph, nx.spring_layout, scale=1, center=(0,0))
gr.node_renderer.glyph = Circle(size=20, fill_color=Spectral4[0])
gr.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width=1)
gr.inspection_policy = NodesAndLinkedEdges()
p.renderers.append(gr)
show(p, notebook_handle=True)

In [8]:
from pyvis.network import Network
from IPython.display import IFrame, HTML, display_html
import os

for node in causal_graph.nodes:
    if node.is_service():
        color = "blue"
        size = 20
    elif node.is_container():
        color = "green"
        size = 10
    elif node.is_middleware():
        color = "purple"
        size = 10
    else:
        color = "grey"
        size = 10
    if node.is_root():
        color = "red"
        size = 25
    causal_graph.nodes[node]["color"] = color
    causal_graph.nodes[node]["size"] = size
    causal_graph.nodes[node]["label"] = node.label

nwg = Network(notebook=True)
relabeled_mapping = mn.MetricNodes.from_list_of_metric_node(list(causal_graph.nodes)).node_to_label()
relabeled_graph = nx.relabel_nodes(causal_graph, relabeled_mapping, copy=True)
nwg.from_nx(relabeled_graph)
nwg.toggle_physics(True)
nwg.show('../tmp/nw.html')